In [1]:
import pandas as pd
import numpy as np
import jupyter_dash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from plotly import graph_objs as go
from dash.dependencies import Input, Output

<ipython-input-1-d8fc588fcbaf>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-d8fc588fcbaf>:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)

In [3]:
#Are stewards (steward activity measured by the ‘steward’ variable) having an impact on the health of trees?

In [4]:
trees['spc_common'] = trees['spc_common'].fillna('Missing')
trees['health'] = trees['health'].fillna('Missing')

In [5]:
q_1_health = pd.DataFrame(trees.groupby(['boroname','spc_common','steward','health'])['tree_id'].count())
q_1_health

tree_id
boroname      spc_common           steward health         
Bronx         American elm         1or2    Good          1
                                   None    Good          1
                                           Poor          2
              Atlantic white cedar 1or2    Good          1
              Callery pear         3or4    Good          1
...                                                    ...
Staten Island sweetgum             None    Good          9
              sycamore maple       None    Good          1
              white oak            1or2    Fair          1
                                   None    Good          1
              willow oak           None    Good          1

[270 rows x 1 columns]

In [6]:
q_1_health_prop = q_1_health.div(q_1_health.sum(level=[0,1]))

In [7]:
q_1_health_prop_un = q_1_health_prop.unstack(level=1)

In [8]:
q_1_health_prop_un

tree_id                             \
spc_common                   American elm American linden Amur maple   
boroname      health steward                                           
Bronx         Good   1or2        0.250000             NaN        NaN   
                     3or4             NaN             NaN        NaN   
                     None        0.250000             NaN        NaN   
              Poor   1or2             NaN             NaN        NaN   
                     None        0.500000             NaN        NaN   
              Fair   1or2             NaN             NaN        NaN   
                     None             NaN             NaN        NaN   
Brooklyn      Good   1or2             NaN        0.250000        NaN   
                     3or4             NaN             NaN        NaN   
                     4orMore          NaN             NaN        NaN   
                     None             NaN        0.600000       1.00   
              Poor   1or2             NaN             NaN        NaN   
                     3or4             NaN             NaN        NaN   
                     None             NaN        0.050000        NaN   
              Fair   1or2             NaN             NaN        NaN   
                     3or4             NaN             NaN        NaN   
                     None             NaN        0.100000        NaN   
Manhattan     Good   1or2        0.333333        0.272727        NaN   
                     3or4             NaN        0.090909        NaN   
                     None             NaN        0.454545        NaN   
              Poor   1or2             NaN             NaN        NaN   
                     None             NaN             NaN        NaN   
              Fair   1or2        0.333333             NaN        NaN   
                     3or4             NaN        0.090909        NaN   
                     None        0.333333        0.090909        NaN   
Queens        Good   1or2        0.500000        0.125000        NaN   
                     3or4             NaN             NaN        NaN   
                     None             NaN        0.375000       0.50   
              Poor   1or2             NaN             NaN        NaN   
                     None             NaN        0.500000        NaN   
              Fair   1or2             NaN             NaN       0.25   
                     3or4             NaN             NaN        NaN   
                     None        0.500000             NaN       0.25   
Staten Island Good   1or2             NaN             NaN        NaN   
                     None             NaN             NaN        NaN   
              Fair   1or2             NaN             NaN        NaN   
                     None             NaN             NaN       1.00   

                                                                \
spc_common                   Atlantic white cedar Callery pear   
boroname      health steward                                     
Bronx         Good   1or2                     1.0          NaN   
                     3or4                     NaN     0.142857   
                     None                     NaN     0.857143   
              Poor   1or2                     NaN          NaN   
                     None                     NaN          NaN   
              Fair   1or2                     NaN          NaN   
                     None                     NaN          NaN   
Brooklyn      Good   1or2                     NaN     0.307692   
                     3or4                     NaN          NaN   
                     4orMore                  NaN          NaN   
                     None                     NaN     0.692308   
              Poor   1or2                     NaN          NaN   
                     3or4                     NaN          NaN   
                     None                     NaN          NaN   
              Fair   1or2      

In [9]:
#fig = px.bar(q_1_health_prop_un, x="boroname", y="health", color="spc_common", barmode="group")
#pd.options.plotting.backend = "plotly"
#fig = q_1_health_prop_un.plot(kind='bar')
#fig.update_layout(barmode='group')
#fig.show()
temp_df = q_1_health_prop_un.reset_index().melt(id_vars=['boroname','steward','health'])
temp_df = temp_df.fillna(0)
temp_df = temp_df[temp_df['health']!='Missing']

In [17]:
#fig = px.bar(temp_df,x='spc_common',y='value',color='boroname')
#fig


In [50]:
health = temp_df.health.unique()
steward = temp_df.steward.unique()
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div([
            html.Label('Health'),
            dcc.Dropdown(
                id='health_dropdown',
                options=[{'label' : x, 'value' : x} for x in health],
                clearable=False,
                value=health[0]
            ),
        ]),
        html.Div([
            html.Label('Steward'),
            dcc.Dropdown(
                id='steward_dropdown',
                options=[{'label': y, 'value': y} for y in steward],
                clearable=False,
                value=steward[0]
            ),
        ]),
    ]),
    dcc.Graph(id='bar-chart', figure=go.Figure())
])


@app.callback(
    Output("bar-chart", "figure"), 
    [Input("health_dropdown", "value"),
     Input("steward_dropdown", "value")])
def update_bar_chart(health,steward):
    fig = px.bar(temp_df[(temp_df["health"] == health) & (temp_df["steward"] == steward) ],x='spc_common',y='value',color='boroname',barmode='group') 
    fig.update_layout(title_text='Species Health-Steward Relationship Across Burroughs')
    return fig

app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Oct/2021 21:20:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:20:32] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:20:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:20:32] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:20:32] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:20:32] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:20:32] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:20:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:20:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:20:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:20:41] "POST /_dash-update-component HTTP/1.1" 